# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [139]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') # доступ к файлу с данными
df.info() # оценка загруженных данных, просмотр регистра, наличия "лишних" пробелов, определение типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [140]:
df.head(10) #просмотр первых 10 строк загруженного файла, ознакомления с названием столбцов и значениями строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<font color='DarkBlue'><b>Выводы по обзору данных</b></font><br>

Видна проблема с колонкой days_employed :

* дни не могут быть вещественным числом
* дни не могут быть отрицательными. Скорее всего проблемы образовались при выгрузке, необходимо создать тикет и добваить в него сотрудника, который сформировал данные, для анализа они нам понадобятся.

Возможно стоит переименовать:

* dob_years в full_years (так логичнее и понятнее при обращении)
* children в children_count (это количество детей, а не бинарный признак)
* education в education_level (уровень образования более логичен)

В столбце education необходимо привести к одному регистру
    
Ежемесячный доход total_incomeскорее всего не сильно нужен с учетом копеек, поэтому возможно стоит перевести в int (целочисленный).

Исходные данные успешно загружены и проанализированы, подробное описание и проблемы по ним указал выше.


### Шаг 2.1 Заполнение пропусков

In [141]:
df.isnull().sum() # поиск пустых значений - 2174 из 21524 строк, получается более 10 %, объем весомый, поэтому необходимо их заполнение. 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [142]:
df[df['total_income'].isnull()].tail()# определим что именно из себя представляют пустые значения
# NAN говорит об отсутсвии данных, NaN замещает отсутствующее в ячейке число и принадлежит к типу float, поэтому с ним можно проводить математические операции.
# Возможные причины появляются пропусков заключаются :
# В колонке "days_employed" дни не могут быть отрицательными. Скорее всего проблемы образовались при выгрузке, необходимо создать тикет и добваить в него сотрудника, который сформировал данные, для анализа они нам понадобятся, но в дальнейшем данный столбец можно будет удалить, если не появится обратной связи от сотрудника, формировавшего данный файл.
# В Колонке "total_income" пропуски могли образоваться из-за неподтверждения официального дохода, например при "серой зарплате", либо неуказанием заемщика, кроме этого, учет более 2-х знаков после запятой не нужен.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


In [143]:
# Восстановим данные в total_income
# Используем медиану, что бы не учитывать "среднюю температуру по всей больнице", некоторые значения доход  сильно выделяются от большинства, поэтому медиана будет точнее, чем определение среднего
# Буду использовать уровень образования и род деятельности для уточнения данных  

import math #нужен для проверки NaN

# calc_median_by_education_and_income вычисляет медиану для income_type по уровню образования и типа работы
# при неопределении среднего сразу по 2-м параметрам - не может быть вычислено (не найдется в основных значениях группы параметров)
# берется среднее по одному - сперва по уровню образования, потом уже по типу сотрудника
def calc_median_by_education_and_income(df, education, income_type):
    # используем оба параметра education и income_type
    median = df[(df['education'] == education) & (df['income_type'] == income_type)]['total_income'].median()
    # если нет совпадения по двум параметрам education_level и income_type, то считаем сначала по education_level, потом по income_type 
    if math.isnan(median):
        median = df[df['education'] == education]['total_income'].median()
        if math.isnan(median):
            median = df[df['income_type'] == income_type]['total_income'].median()    
    return median

# перебираем все строки стобцов education и income_type для пустых значений
# только для пустых значенй используем медиану, соответствующую их группе
for education_level in df[df['total_income'].isnull()]['education'].unique():
    for income_type in df[df['total_income'].isnull()]['income_type'].unique():
        median = calc_median_by_education_and_income(df, education_level, income_type)
        searchRow = (df['education'] == education_level) & (df['income_type'] == income_type) & (df['total_income'].isnull())
        df.loc[searchRow,'total_income'] = df.loc[searchRow,'total_income'].fillna(median)

# проверяем, что все писправлено и в столбце  income_type  не осталось пустых значений, столбец days_employed был удален.
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [144]:
# Был проведен анализ возникновения аномалий:
# отрицательны значения могли появиться из-за не верной формулы в подсчете стажа, 
# вычли из даты начала стажа текущую дату, а не наоборот. 
# Слишком высокие значения у пенсионеров наверное из-за того что раньше данные хранились в другом формате 
# и при переходе на новую БД могли не заметить ошибку, но это всего лишь предположения.
# Нулевой возраст можно заполнить медианным значением в этой группе занятости. 
# Попадаются строчки с отрицательным количество детей ‘-1’. 
# Есть одна строка с третьим полом, не мужчина и не женщина. 
# Если бы стояла задача искать зависимость от стажа то нужно было бы точно запрашивать верные данные, 
# потому что из-за предположений можно сильно изменить картину.

# столбец с днями трудового стажа сожержит неверную информацию на которую мы "объективно" повлиять или дополнить не можем 
# (даже при переводе ее в дни, получается, что трудовой стаж может быть больше возраста)
# кроме этого в трудовом стаже присутствуют отрицательных значения, которые можно сделать положительными 
# при помощи функции abs, но количество отработанных дней при этом все равно может первышать возраст участников
# для ответов на поставленные задачи данный столбец нам не нужен, поэтому принято решенрие его удалить

del df['days_employed']

df.isnull().sum()

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Обработка других ошибок в данных

Дополнительно поизучав данные на уровне подсчета уникальных значений (value_counts()) для каждого столбца, был выявлен еще ряд проблем:

children содержит значение 20 и -1, хотя отрицательного числа быть не может и 20 сильно выделяется на фоне остальной выборки

full_years содержит нулевой возраст
gender содержит пол XNA

purpose содержит опечатку - ремонт жильЮ
Про возможные причины и действия:

значение 20 в количестве детей скорее всего опечатка, когда набирая 2, случайно задели ноль

значение -1 могло означать раннее отсуствие детей либо их наличие, но почеуму-то выгруженное, как отрицательное число. Т.к. по df[df['children'] == -1]['family_status'].value_counts() можно убедиться, что большая часть этих людей находится или находилась в браке, остановимся на втором варианте.

нулевой возраст быть не может, скорее всего он нам просто неизвестен. Т.к. в нашем случае вопросов касательно возраста нет, ничего не делаем.

поскольку пол XNA только один, а в наших данных большинство женщины - заменим значением по умолчанию - F
учитывая количество строк, ошибка в самой категории на сайте или системе. У себя исправим, ответственных людей предупредим.

Исправим эти моменты.

In [145]:
df.loc[df['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья' # исправим опечатку в purpose
df.loc[df['purpose'] == 'ремонт жилью']['purpose'].count() # проверим, что нет ошибок

0

In [146]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'F' # исправим значение поля 'gender'
df.loc[df['gender'] == 'XNA']['gender'].count() # проверим, что нет ошибок

0

In [147]:
df.loc[df['children'] == 20, 'children'] = 2 # исправим значения 20 для возраста ('children') согласно нашей гипотезе
df.loc[df['children'] == 20]['children'].count() # проверим, что нет ошибок

0

In [148]:
df.loc[df['children'] == -1, 'children'] = 1 # исправим значение -1 для количества детей 'children' согласно нашей гипотезы (т.е. заменим на 1)
df.loc[df['children'] == -1]['children'].count() # проверим, что нет ошибок

0

Пропущенным значениям дохода (total_income) мы выставили медиану, но не всей выборки, а только по конкретным категориям, к которым люди относились. Это позволит получить более точную информацию и снизить возможную статистическую погрешность.

Пропуски столбца days_employed не быи учтены т.к. они не несут для нас значимой информации и заменить информацию в них нечем.

Причины возникновения пропусков - скорее всего они связаны с тем, что у людей нет подтвержденного дохода или стажа. Сейчас у нас нет возможности уточнить этот момент у коллег, делающих выгрузку.

В данных встречаются артефакты и нам стоит об этом предупредить тех, кто делал выгрузку. Однако их количество вряд ли отразится на статических результатах исследования. Тем не менее, для большей точности, ключевые моменты были исправлены.

### Шаг 2.3. Изменение типов данных.

In [149]:
df['total_income'] = df['total_income'].astype('int') # Замена вещественного тип данных в столбце total_income на целочисленный
df.info() # проверка замены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Шаг 2.4. Удаление дубликатов.

In [150]:
df['education'] = df['education'].str.lower() # приведем education к одному регистру для более удобного анализа
df.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [151]:
df.duplicated().sum() # проверка на дубликаты при помощи duplicated (),
# использован данный метод, так как при проверке value_counts() нужно  создавать отдельную Series, 
# которая в дальнейшем не будет использоваться 

54

In [152]:
df.drop_duplicates(subset=None, keep="first", inplace=True) # удалим найденные дубликаты
df.duplicated().sum() # поверка что дубликаов не осталось

0

Дубликатов оказалось не так много и их удалось быстро удалить. Для удобства работы с данными столбец education был привден к одному регистру.
В других столбцах подобных проблем замечено не было, выборка проводилась с начала и конца таблицы при помощи head() и tail()
Повторений информации больше замечено не было.


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [153]:
df_education = df[['education_id','education']]#  создание нового датафрейма со столбцами education_id и education
df_education = df_education.drop_duplicates().reset_index(drop=True)# удалим дубликаты из словаря
df_education.head() #проверка созданного df

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [154]:
df_family_status = df[['family_status_id','family_status']] #  создание нового датафрейма со столбцами family_status_id и family_status
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True)# удалим дубликаты из словаря
df_family_status.head() #проверка созданного df

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [155]:
del df['education'] # удалим education из df, оставив его идентификаторы education_id 
df.head()

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [156]:
del df['family_status'] # удалим family_status из df, оставив его идентификаторы family_status_id 
df.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Лемы

In [157]:
# выделим леммы для столбца с целями получения кредита 'purpose'

#подключим PyMystem
from pymystem3 import Mystem
m = Mystem()

# получим леммы для каждой причины, чтобы дальше проанализировать
# используем словарь, а не список, чтобы дальше это использовать для категоризации
lemmas_dict = dict()
for purpose in df['purpose'].unique():
    lemmas_dict[purpose] = m.lemmatize(purpose)
    
# анализируем леммы - удалим пробелы, выделим самые часто встречаемые слова
# отсортируем по частоте встречания слов, чтобы получить словарь основных причин
# те слова, что встречаются только один раз, удалим

#сначала определим частоту встречания слов по всем причинам кредита
lemmas_count = dict()
for key, lemmas in lemmas_dict.items():
    for lemma in lemmas:
        if lemma != ' ' and lemma != '\n':
            if not lemma in lemmas_count:
                lemmas_count[lemma] = 1
            else:
                lemmas_count[lemma] += 1
                
#выведем самые популярные встречаемые слова              
for k in sorted(lemmas_count, key=lemmas_count.get, reverse=True):
    if lemmas_count[k] > 2:
        print(k, lemmas_count[k])

покупка 10
недвижимость 10
автомобиль 9
образование 9
жилье 7
с 5
операция 4
на 4
свой 4
свадьба 3
строительство 3
получение 3
высокий 3


Видим явное деление на 4 группы:

Недвижимость (его аналог жилье) 

Автомобиль

Образование

Свадьба

Леммы позволят нам поделить все цели кредитов по соответствующим группам.

### Шаг 2.6. Категоризация дохода.

In [158]:
df['total_income'].value_counts() # подсчет количества клиентов с разным уровнем месячного дохода

136818    731
114483    291
159652    272
166164    207
201911    177
         ... 
390148      1
113661      1
111612      1
199675      1
264193      1
Name: total_income, Length: 18626, dtype: int64

In [159]:
# На основании диапазонов, указанных ниже, создадим новый столбец total_income_category с категориями:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'.
# Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

def make_income_category(total_income_category):
    if total_income_category < 30000:
        return 'E'
    if 30001 <= total_income_category < 50000:
        return 'D'
    if 50001 <= total_income_category < 200000:
        return 'C'
    if 200001 <= total_income_category < 1000000:
        return 'B'
    return 'A'


df['total_income_category'] = df['total_income'].apply(make_income_category)

# проверим, что все ок
df['total_income_category'].value_counts()

C    15850
B     5224
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Анализируя полученные данные можно сделать вывод, что кредит практически не интересен 2-м категориям граждан, кто зарабатывает в месяц много (более 1 миллиона рублей и кто зарабатывает мало - менее 30 000 рублей). Больше всего нуждаются в кредитных деньгах люди с доходом от 50 000 до 200 000 рублей в месяц, так называемый "средний класс"

### Шаг 2.7. Категоризация целей кредита.

In [160]:
# посмотрим распределение по целям кредитов
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилья                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [161]:
# Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category,
# в который войдут следующие категории:
# 'операции с автомобилем',
# 'операции с недвижимостью',
# 'проведение свадьбы',
# 'получение образования'.

# сделаем столбец с причинами для взятия кредита по категориям, что мы выделили на этапе лемматизации

# напишем функцию, по которой заполним столбец
def make_true_purpose(purpose_category):
    #lemmas_dict мы сохраняли ранее
    purpose_lemmas = lemmas_dict[purpose_category]
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'операции с недвижимостью'
    
    if 'автомобиль' in purpose_lemmas:
        return 'операции с автомобилем'
    
    if 'образование' in purpose_lemmas:
        return 'получение образования'
    
    if 'свадьба' in purpose_lemmas:
        return 'проведение свадьбы'
    
    return ''

df['purpose_category'] = df['purpose'].apply(make_true_purpose)
    
df[df['purpose_category'] == ''] #провека, что никто не остался без категории

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


In [162]:
df['purpose_category'].value_counts() # определим распределение целей по кредитам

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

In [163]:
# Поскольку больше двух детей данных в принципе не очень много, имеет смысл объеденить их в одну категорию

def make_children_category(children_count):
    if children_count == 1:
        return '1 ребенок'
    if children_count == 2:
        return '2 ребенка'
    if children_count >= 3:
        return 'Много детей'
    return 'Нет детей'

df['children'] = df['children'].apply(make_children_category)

# проверим, что все ок
df['children'].value_counts()

Нет детей      14107
1 ребенок       4856
2 ребенка       2128
Много детей      380
Name: children, dtype: int64

### Ответы на вопросы.

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок ?

In [164]:
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
children,,,
Нет детей,1063,14107,7.54%
Много детей,31,380,8.16%
1 ребенок,445,4856,9.16%
2 ребенка,202,2128,9.49%


### Вывод 1:

Влияние наличия детей на вероятность просрочки платежа

По наличию детей

Нет детей - 7.54%

Много детей - 8.16%

1 ребенок - 9.17%

2 ребенка - 9.49%

Абсолютная разница между минимальным и максимальным значением - 1.95%. 

Анализируя данные можно сделать вывод, что меньше всего вероятность просрочки по кредиту для клиентов с двумя детьми.

### Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок ?

In [165]:
df_with_family_status = df.merge(df_family_status, on='family_status_id', how='left') # возвращение family_status в DF
# построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря)
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt')) # сортировка и использование make_proportion
data_pivot

,sum,count,make_proportion
,debt,debt,debt
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12344,7.54%
гражданский брак,388,4163,9.32%
Не женат / не замужем,274,2810,9.75%


### Вывод 2:

Стоит отметить, что вдовец/вдова и люди в разводе имеют наименьший риск просрочки. Возможно, это связно с тем, что кредит берется более осознанно.

Самая "опасная" категория - люди не женатые и состоящие в гражданском браке.

### Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок ?

In [170]:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'.

# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot


,sum,count,make_proportion
,debt,debt,debt
total_income_category,,,
D,21,350,6.0%
B,364,5224,6.97%
A,2,25,8.0%
C,1352,15850,8.53%
E,2,22,9.09%


### Вывод 3:

Меньше всего просрочек у категорий с доходом от 30 000  до 50 000 рублей и от 200 000 до 1 000 000 рублей, больше всего просрочек у группы с доходом от 50 001 до 200 000 рублей и менее 30 000 рублей.

### Вопрос 4. Как разные цели кредита влияют на его возврат в срок ?

In [172]:
# построим сводную таблицу для ответа на вопрос
data_pivot = df.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
data_pivot

,sum,count,make_proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10814,7.23%
проведение свадьбы,186,2335,7.97%
получение образования,370,4014,9.22%
операции с автомобилем,403,4308,9.35%


### Вывод 4:

Меньше всего просрочек у людей, взявших деньги у банка на покупку квартиры и празднование свадьбы, а по кредитам, взятым на образование и покупку автомобилей можно отметить самый высокий процент просрочек.

## Общий вывод:

Идеальный заемщик для банка -  человек без детей, состоящий или состоявший в официальных отношениях, имеющий уровень дохода выше среднего и берущий деньги на недвижимость.

"Разброс" просроченных выплат по каждой из категорий примерно одинаковый. В среднем, это диапазоны от 7 до 9 процентов. В остальном разница в абсолютных цифрах не такая большая.

Вывод по категориям:

1)Влияние наличия детей на вероятность просрочки платежа

По наличию детей

Нет детей - 7.54%

Много детей - 8.16%

1 ребенок - 9.17%

2 ребенка - 9.49%

Абсолютная разница между минимальным и максимальным значением - 1.95%.

Анализируя данные можно сделать вывод, что меньше всего вероятность просрочки по кредиту для клиентов с двумя детьми.

2)Вдовец/вдова и люди в разводе имеют наименьший риск просрочки. 

Самая "опасная" категория - люди не женатые и состоящие в гражданском браке.

3)Меньше всего просрочек у категорий с доходом от 30 000  до 50 000 рублей и от 200 000 до 1 000 000 рублей, больше всего просрочек у группы с доходом от 50 001 до 200 000 рублей и менее 30 000 рублей.

4)Меньше всего просрочек у людей, взявших деньги у банка на покупку квартиры и празднование свадьбы, а по кредитам, взятым на образование и покупку автомобилей можно отметить самый высокий процент просрочек.